In [1]:
import time
import os
import math
import tushare as ts
from datetime import datetime
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from IPython.display import clear_output

sys.path.append("../utils/")
import date_util
import token_util
import enter_util
import stock_util

### 提取数据

In [2]:
# trade_date = "20190830"
# delta_price = 30.0
# stock_list = stock_util.get_stock_list(trade_date, delta_price)

### 选出符合入场点的股票

In [3]:
test_choose_list = enter_util.get_condidate_stock('20191021')

In [4]:
test_choose_list

,stock_code,name,pct_chg_short,his_macd_discount,amount_mean_20,amount_discount_mean,rank_factor
0,000607.SZ,华媒控股,-0.099093,0.812091,0.226051,2.073715,1.912428
1,000662.SZ,天夏智慧,2.525744,-0.242129,0.707177,-0.268959,1.239852
2,002579.SZ,中京电子,-0.564286,0.914231,1.461202,-0.542892,1.091243
3,002687.SZ,乔治白,-0.687288,1.311954,-1.406657,0.296284,0.413124
4,603136.SH,天目湖,-0.760834,0.068017,-0.568280,0.957470,-0.117805
5,002250.SZ,联化科技,-0.450129,-0.330379,0.502791,-0.445375,-0.526736
6,002528.SZ,英飞拓,0.025967,0.079989,-0.461798,-1.251568,-0.763710
7,600818.SH,中路股份,0.142451,-0.535928,-1.334776,-0.071543,-1.167862
8,603797.SH,联泰环保,-0.132531,-2.077846,0.874289,-0.747131,-2.080533


# TEST